## GTFS Realtime - TripUpdate

| **Field**               | **Supported** | **Comments**                                                                 |
|-------------------------|---------------|-----------------------------------------------------------------------------|
| **trip (TripDescriptor)**|               |                                                                             |
| trip_id                 | Yes           |                                                                             |
| route_id                | Yes           |                                                                             |
| direction_id            | Yes           | If available in the static feed.                                             |
| start_time              | Yes           |                                                                             |
| start_date              | Yes           |                                                                             |
| schedule_relationship   | Yes           | Supported values are: SCHEDULED, CANCELED                                    |
| **vehicle (VehicleDescriptor)** |       |                                                                             |
| id                      | Yes           |                                                                             |
| label                   | Yes           | trip_headsign if available in the static feed.                               |
| license_plate           | Yes           | Availability not guaranteed.                                                 |
| **stop_time_update (StopTimeUpdate)** | Yes |                                                                             |
| stop_sequence           | No            |                                                                             |
| stop_id                 | Yes           |                                                                             |
| **arrival (StopTimeEvent)** |           |                                                                             |
| delay                   | No            |                                                                             |
| time                    | Yes           |                                                                             |
| uncertainty             | No            |                                                                             |
| **departure (StopTimeEvent)** |         |                                                                             |
| delay                   | No            |                                                                             |
| time                    | Yes           |                                                                             |
| uncertainty             | No            |                                                                             |
| schedule_relationship   | No            | Supported values are: SCHEDULED, NO_DATA if stop is closed, SKIPPED if partial trip cancellation |
| timestamp               | Yes           |                                                                             |
| delay                   | No            |                                                                             |


### Ce script récupère en temps réel les positions des véhicules de transport public à Tampere via l'endpoint vehicleposition de l'API GTFS Realtime de Waltti. 

In [ ]:
import os
import base64
import requests
import json
from google.transit import gtfs_realtime_pb2  # GTFS Realtime
from google.protobuf.json_format import MessageToJson
from confluent_kafka import Producer

# Définir les variables d'environnement
os.environ["CLIENT_ID"] = "6722072041100899"  # Remplacez par votre véritable Client ID
os.environ["CLIENT_SECRET"] = "ISo0oqPMkfCB8wvaPSxrfj20EmnuvKwE"  # Remplacez par votre véritable Client Secret

# Vérification des variables d'environnement
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

if not client_id or not client_secret:
    print("Error: CLIENT_ID and CLIENT_SECRET must be set as environment variables.")
    exit()

# Authentification avec Basic Authentication
auth_string = f"{client_id}:{client_secret}"
encoded_auth = base64.b64encode(auth_string.encode()).decode()

# Préparation de l'URL et des en-têtes
city = "tampere"  # Remplacez par votre ville
endpoint = "vehicleposition"  # Endpoint choisi : tripupdate, vehicleposition, ou servicealert
url = f"https://data.waltti.fi/{city}/api/gtfsrealtime/v1.0/feed/{endpoint}"

headers = {
    "Authorization": f"Basic {encoded_auth}",
}

# Configuration du producteur Kafka
kafka_config = {
    'bootstrap.servers': 'localhost:9092',  # Adresse du broker Kafka
    'client.id': 'python-producer'
}
producer = Producer(kafka_config)
kafka_topic = "gtfs_realtime_topic"  # Remplacez par votre topic Kafka

# Callback pour la livraison Kafka
def delivery_report(err, msg):
    if err is not None:
        print(f"Message delivery failed: {err}")
    else:
        print(f"Message delivered to {msg.topic()} [partition: {msg.partition()}]")

# Faire la requête API
try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Lève une exception pour les erreurs HTTP

    # Parsing des données Protobuf et conversion en JSON
    feed = gtfs_realtime_pb2.FeedMessage()
    feed.ParseFromString(response.content)
    feed_json = json.loads(MessageToJson(feed))

    # Afficher les données localement
    print(json.dumps(feed_json, indent=4))

    # Envoyer les données à Kafka
    for entity in feed.entity:
        vehicle_json = json.loads(MessageToJson(entity))
        producer.produce(
            kafka_topic,
            json.dumps(vehicle_json),
            callback=delivery_report
        )

    # Assurez-vous que tous les messages sont bien envoyés
    producer.flush()

except requests.exceptions.RequestException as e:
    print(json.dumps({"error": f"Error connecting to the API: {e}"}, indent=4))
except Exception as e:
    print(json.dumps({"error": f"Unexpected error: {e}"}, indent=4))


{
    "header": {
        "gtfsRealtimeVersion": "2.0",
        "incrementality": "FULL_DATASET",
        "timestamp": "1732737985"
    },
    "entity": [
        {
            "id": "10299_378",
            "vehicle": {
                "trip": {
                    "tripId": "73_15525_8731163",
                    "startTime": "21:40:00",
                    "startDate": "20241127",
                    "scheduleRelationship": "SCHEDULED",
                    "routeId": "610299",
                    "directionId": 0
                },
                "position": {
                    "latitude": 61.49891,
                    "longitude": 23.772165,
                    "bearing": 260.0,
                    "speed": 0.8055556
                },
                "currentStopSequence": 28,
                "currentStatus": "IN_TRANSIT_TO",
                "timestamp": "1732737985",
                "congestionLevel": "UNKNOWN_CONGESTION_LEVEL",
                "stopId": "0506",
              

%6|1732739944.196|FAIL|python-producer#producer-2| [thrd:Ala.:9092/0]: Ala.:9092/0: Disconnected (after 1840485ms in state UP)
%3|1732739944.196|FAIL|python-producer#producer-2| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Connect to ipv4#127.0.0.1:9092 failed: Connection refused (after 0ms in state CONNECT)
%3|1732739944.940|FAIL|python-producer#producer-2| [thrd:Ala.:9092/0]: Ala.:9092/0: Connect to ipv4#127.0.1.1:9092 failed: Connection refused (after 0ms in state CONNECT)
%3|1732739945.941|FAIL|python-producer#producer-2| [thrd:Ala.:9092/0]: Ala.:9092/0: Connect to ipv4#127.0.1.1:9092 failed: Connection refused (after 0ms in state CONNECT, 1 identical error(s) suppressed)
%3|1732739946.941|FAIL|python-producer#producer-2| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Connect to ipv4#127.0.0.1:9092 failed: Connection refused (after 0ms in state CONNECT, 1 identical error(s) suppressed)
%3|1732739978.962|FAIL|python-producer#producer-2| [thrd:localhost:9092

### servicealert

In [ ]:
import os
import base64
import requests
import json
from google.transit import gtfs_realtime_pb2  # GTFS Realtime
from google.protobuf.json_format import MessageToJson
from confluent_kafka import Producer

# Définir les variables d'environnement
os.environ["CLIENT_ID"] = "6722072041100899"  # Remplacez par votre véritable Client ID
os.environ["CLIENT_SECRET"] = "ISo0oqPMkfCB8wvaPSxrfj20EmnuvKwE"  # Remplacez par votre véritable Client Secret

# Vérification des variables d'environnement
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

if not client_id or not client_secret:
    print("Error: CLIENT_ID and CLIENT_SECRET must be set as environment variables.")
    exit()

# Authentification avec Basic Authentication
auth_string = f"{client_id}:{client_secret}"
encoded_auth = base64.b64encode(auth_string.encode()).decode()

# Préparation de l'URL et des en-têtes
city = "tampere"  # Remplacez par votre ville
endpoint = "servicealert"  # Endpoint choisi : tripupdate, vehicleposition, ou servicealert
url = f"https://data.waltti.fi/{city}/api/gtfsrealtime/v1.0/feed/{endpoint}"

headers = {
    "Authorization": f"Basic {encoded_auth}",
}

# Configuration du producteur Kafka
kafka_config = {
    'bootstrap.servers': 'localhost:9092',  # Adresse du broker Kafka
    'client.id': 'python-producer'
}
producer = Producer(kafka_config)
kafka_topic = "gtfs_realtime_topic"  # Remplacez par votre topic Kafka

# Callback pour la livraison Kafka
def delivery_report(err, msg):
    if err is not None:
        print(f"Message delivery failed: {err}")
    else:
        print(f"Message delivered to {msg.topic()} [partition: {msg.partition()}]")

# Faire la requête API
try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Lève une exception pour les erreurs HTTP

    # Parsing des données Protobuf et conversion en JSON
    feed = gtfs_realtime_pb2.FeedMessage()
    feed.ParseFromString(response.content)
    feed_json = json.loads(MessageToJson(feed))

    # Afficher les données localement
    print(json.dumps(feed_json, indent=4))

    # Envoyer les données à Kafka
    for entity in feed.entity:
        vehicle_json = json.loads(MessageToJson(entity))
        producer.produce(
            kafka_topic,
            json.dumps(vehicle_json),
            callback=delivery_report
        )

    producer.flush()

except requests.exceptions.RequestException as e:
    print(json.dumps({"error": f"Error connecting to the API: {e}"}, indent=4))
except Exception as e:
    print(json.dumps({"error": f"Unexpected error: {e}"}, indent=4))


{
    "header": {
        "gtfsRealtimeVersion": "2.0",
        "incrementality": "FULL_DATASET",
        "timestamp": "1733306734"
    },
    "entity": [
        {
            "id": "14196",
            "isDeleted": false,
            "alert": {
                "activePeriod": [
                    {
                        "start": "1721703618",
                        "end": "1734731958"
                    }
                ],
                "informedEntity": [
                    {
                        "stopId": "3569"
                    }
                ],
                "cause": "CONSTRUCTION",
                "effect": "NO_SERVICE",
                "url": {
                    "translation": [
                        {
                            "text": "",
                            "language": "en"
                        },
                        {
                            "text": "",
                            "language": "fi"
                        }
       